In [ ]:
!pip install pysatml

  Preparing metadata (setup.py) ... done
  Created wheel for pysatml: filename=pysatml-0.1.2-py3-none-any.whl size=6552 sha256=572744202a5f0530628b38a6f1b9d803a116faa72ce4ca776353332863dc26a8
  Stored in directory: /root/.cache/pip/wheels/06/5d/37/57c977ec17ac368135ae43ca725e2186ddd5234ccc6c244933
Successfully built pysatml


In [ ]:
!pip install ipyparallel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00


In [45]:
# numeric packages
import numpy as np
import pandas as pd

# filesystem and OS
import sys, os, time
import glob

# plotting
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

# compression
import gzip
import pickle
import copy

# widgets and interaction
from IPython.display import display, clear_output

import warnings
warnings.filterwarnings('ignore')


In [46]:
# numeric packages
import numpy as np
import pandas as pd

# filesystem and OS
import sys, os, time
import glob

# plotting
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
from IPython.display import display, clear_output

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

# these magics ensure that external modules that are modified are also automatically reloaded
# %reload_ext autoreload
# %autoreload 2

In [47]:
outPath = "/home/data/urban-atlas/extracted-data"

if not os.path.exists(outPath):
    os.makedirs(outPath)

In [48]:
locations_path = "/home/data/urban-atlas/extracted-data/"

grid_location_files = glob.glob("%s/*/sample_locations_raster_25km.csv"%locations_path)
grid_location_files = {f.split("/")[-2]:f for f in grid_location_files}

more_location_files = glob.glob("%s/*/additional_sample_locations.csv"%locations_path)
more_location_files = {f.split("/")[-2]:f for f in more_location_files}

In [43]:
def load_locations(path="./"):
    '''
    This function loads csv files for which each line contains information on the
    location of an image to be acquired from Google Maps.
    '''


    # Load the CSV files
    grid_locations_df = pd.read_csv(path + "/sample_locations_raster_25km.csv")
    more_locations_df = pd.read_csv(path + "/additional_sample_locations25km.csv")

    # Add 'ITEM' column if it does not exist
    if 'ITEM' not in more_locations_df.columns:
        more_locations_df['ITEM'] = 'default_value'  # Use a default value or some logic to assign values

    # Now rename 'ITEM' to 'class'
    more_locations_df = more_locations_df.rename(columns={"ITEM": "class"})

    # Print the counts of samples in each dataframe
    print(f"Grid samples: {len(grid_locations_df)}. Additional samples: {len(more_locations_df)}")

    # Initialize missing columns in more_locations_df
    more_locations_df['grid-i'] = np.nan
    more_locations_df['grid-j'] = np.nan

    # Define the columns to be used
    columns = ["lon", "lat", "grid-i", "grid-j", "class"]

    # Concatenate both dataframes based on specified columns
    locations = pd.concat([grid_locations_df[columns], more_locations_df[columns]])

    # Reset the index and drop the old index column
    locations = locations.reset_index(drop=True)

    return locations

In [49]:
city = "madrid, es"

workdir = "%s/%s" % (outPath, city)
os.chdir(workdir)

locations = load_locations()
print(len(locations))

locations.groupby("class").apply(len)

Grid samples: 7840. Additional samples: 26948
34788


class
Agricultural + Semi-natural areas + Wetlands                     4247
Construction sites                                                 62
Continuous Urban Fabric (S.L. > 80%)                              152
Discontinuous Dense Urban Fabric (S.L. : 50% -  80%)              129
Discontinuous Low Density Urban Fabric (S.L. : 10% - 30%)         153
Discontinuous Medium Density Urban Fabric (S.L. : 30% - 50%)      205
Discontinuous Very Low Density Urban Fabric (S.L. < 10%)           29
Fast transit roads and associated land                             30
Forests                                                           114
Green urban areas                                                 353
Industrial, commercial, public, military and private units        167
Mineral extraction and dump sites                                   4
Other roads and associated land                                  2074
Railways and associated land                                       18
Sports and lei

In [ ]:
!pip install --upgrade pip setuptools wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:00
  Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [4]:
!git clone https://github.com/adrianalbert/satapi.git

Cloning into 'satapi'...
remote: Enumerating objects: 265, done.
remote: Total 265 (delta 0), reused 0 (delta 0), pack-reused 265
Receiving objects: 100% (265/265), 3.74 MiB | 6.68 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [8]:
!ls

sample_data  satapi


In [15]:
!cd satapi/

In [18]:
!cd satapi/
!python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [35]:
import pysatapi
from pysatapi.googlemaps.GoogleMaps import GoogleMaps

In [36]:
import os
from google.colab import userdata


googleAPIKey = userdata.get('API_VALUE')

if googleAPIKey is None:
    raise ValueError("API key is not set. Please set it in the secrets tab.")

gmClient = GoogleMaps(key=googleAPIKey)

MAX_REQUESTS = 25000
MAX_TRIES    = 2
img_size     = 224
ZOOM         = 17

In [38]:
cities = ["madrid"]

In [40]:
def download_images(locations, prefix="", out_path="./"):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    global n_requests

    for i,r in locations.iterrows():
        clear_output(wait=True)
        print("Pulling image %d/%d... (# API requests = %d)"%(i,len(locations), n_requests))
        label, lat, lon, grid_i, grid_j = r['class'], r['lat'], r['lon'], r['grid-i'], r['grid-j']

        basename = "%s/%s/%s_z%d_%2.5f_%2.5f"%(out_path, label, prefix, ZOOM, lat, lon)
        if not np.isnan(grid_i) and not np.isnan(grid_j):
            cur_filename = "%s_grid-i%d_grid-j%d.jpg"%(basename, grid_i, grid_j)
        else:
            cur_filename = "%s.jpg"%basename
        print(cur_filename)

        if os.path.exists(cur_filename):
            continue

        req = gmClient.construct_static_url((lat,lon), maptype="satellite", zoom=ZOOM, \
                                            imgsize=(int(img_size*1.18), int(img_size*1.18)))
        img = gmClient.get_static_map_image(req, filename=cur_filename, \
                                            max_tries=MAX_TRIES,\
                                            crop=True)

        if img is None or n_requests >= MAX_REQUESTS:
            print("API requests quota exceeded!")
            break
        n_requests += 1

        # display samples every now and then
        if i % 100 == 0:
            plt.imshow(img)
            plt.title("image %d (label = %s)"%(i,label))
            plt.show()
            time.sleep(5)

In [41]:
n_requests = 0

In [58]:
!pip install requests pillow scikit-image

In [59]:
# path to save data

extraction_path = "/home/data/urban-atlas/imagery/"
if not os.path.exists(extraction_path):
    os.makedirs(extraction_path)

# download data for a given city

city = "madrid, es"

locations = load_locations("%s/%s" % (outPath, city))

download_images(locations, prefix=city.replace(" ","-"), out_path="%s/%s"%(extraction_path, city))

Pulling image 0/34788... (# API requests = 0)
/home/data/urban-atlas/imagery//madrid, es/Agricultural + Semi-natural areas + Wetlands/madrid,-es_z17_40.45484_-4.01033_grid-i0_grid-j33.jpg


AttributeError: module 'requests' has no attribute 'fetch'